In [11]:
from pymongo import MongoClient
from sshtunnel import SSHTunnelForwarder
import pprint
from bson.objectid import ObjectId
import pyperclip as clip
import json
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools as it
# from datasu import auc
# import pixiedust
# clip.copy(json.dumps(m1['msg']))
import nbimporter
from common_functions import *

Importing Jupyter notebook from common_functions.ipynb


### funcs

### load data

In [26]:
# MONGO_HOST = "ec2-52-23-187-115.compute-1.amazonaws.com"
MONGO_DB = "marketpulzz"
# server = SSHTunnelForwarder(
#     MONGO_HOST,
#     ssh_username='ubuntu',
#     ssh_pkey="/Users/sasha/.ssh/mmuze.pem",
# #     ssh_private_key_password="secret",
#     remote_bind_address=('127.0.0.1', 27017),
#     local_bind_address=('127.0.0.1', 63328),
#     set_keepalive = 5,
# )

# server.start()
# client = MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port
# db = client[MONGO_DB]

In [27]:
client_local = MongoClient('127.0.0.1', 27017)
db_local = client_local[MONGO_DB]

In [28]:
messages = list(db_local.genie_conversation_messages_emb.find({}))

In [29]:
texts = list(map(lambda m: m['msg']['text'], messages))

In [30]:
lengths = np.array(list((map(lambda t: len(t.split(' ')), texts))))

In [31]:
lengths.mean(), lengths.max(), lengths.min(), 

(19.45244311823946, 198, 1)

In [32]:
np.histogram(lengths)

(array([1818,  618,  160,   43,   26,   11,    3,    1,    0,    1]),
 array([  1. ,  20.7,  40.4,  60.1,  79.8,  99.5, 119.2, 138.9, 158.6,
        178.3, 198. ]))

### embed messages

In [33]:
def add_emb_to_message(message, field_name, vector, metadata):
    current_embs = message.get('embs', {})
    field_embs = current_embs.get(field_name, [])   
    embd_data = {**metadata, 'vector':vector.tolist()}    
    field_embs.append(embd_data)
    current_embs[field_name] = field_embs
    message['embs'] = current_embs
    return message

#### BERT

In [25]:
#bert-serving-start -model_dir /Users/sasha/dev/mmuze/_models/BERT/uncased_L-12_H-768_A-12/ -num_worker=1 -max_batch_size 16 -max_seq_len 200 -pooling_strategy REDUCE_MEAN -pooling_layer -2 
metadata = {            
    'version': '1.0',
    'encoder': 'BERT',
    'model': 'uncased_L-12_H-768_A-12',    
    'max_seq_len': 200,
    'pooling_strategy': 'REDUCE_MEAN', 
    'pooling_layer': '-5',
#     'dimensions': emb_bert[0].shape[0]
}

print(f"bert-serving-start \
-model_dir /Users/sasha/dev/mmuze/_models/BERT/{metadata['model']} \
-num_worker=1 \
-max_batch_size 16 \
-max_seq_len {metadata['max_seq_len']} \
-pooling_strategy {metadata['pooling_strategy']} \
-pooling_layer {metadata['pooling_layer']}")

bert-serving-start -model_dir /Users/sasha/dev/mmuze/_models/BERT/uncased_L-12_H-768_A-12 -num_worker=1 -max_batch_size 16 -max_seq_len 200 -pooling_strategy REDUCE_MEAN -pooling_layer -5


In [34]:
from bert_serving.client import BertClient
bc = BertClient()

In [35]:
%%time
emb_bert = bc.encode(texts)

CPU times: user 139 ms, sys: 48.5 ms, total: 187 ms
Wall time: 11min 38s


In [36]:
messages_emb = list(map(lambda mess_emb: add_emb_to_message(mess_emb[0], 'text', mess_emb[1], metadata), zip(messages.copy(),emb_bert)))

In [37]:
for m in messages_emb:
    db_local.genie_conversation_messages_emb.save(m, check_keys=False)

/usr/local/opt/pyenv/versions/anaconda3-5.3.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: save is deprecated. Use insert_one or replace_one instead
  


In [24]:
e = messages_emb[0]['embs']['text']

#### USE

In [ ]:
# https://stackoverflow.com/questions/48092772/add-operation-to-graph-without-with-as-clause
import tensorflow_hub as hub
graph = tf.Graph()
cm = graph.as_default()   
cm.__enter__()
# module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" 
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url, trainable=True)

session = tf.Session(graph=graph)
session.run(tf.global_variables_initializer())
session.run(tf.tables_initializer())
input1 = tf.placeholder(tf.string, shape=(None))
emb = embed(input1)


def encode_USE(texts):   
    emb1 = session.run([emb], feed_dict={ input1: texts })
    return emb1

#warm up
_ = encode_USE(["sasha", "miko"])

In [ ]:
%%time
text_emb_USE = encode_USE(texts)[0]

In [ ]:
metadata = {            
    version: '1.0'
    encoder: 'USE',
    model: 'uncased_L-24_H-1024_A-16',    
    max_seq_len: 200,
    pooling_strategy: 'REDUCE_MEAN' 
    pooling_layer: [-1, -2],
}

In [ ]:
messages_emb = list(map(lambda mess,emb: add_emb_to_message(mess, 'text', emb, metadata), zip(messages,text_emb_USE)))

In [ ]:
for(m in messages_emb):
    db_local.genie_conversation_messages_emb.save(m, check_keys=False)